## Data Cleaning and cleansing

 First import all important modules

In [5]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import ssl
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from urllib.parse import urlparse 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Set the path to the CA certificates bundle
ssl._create_default_https_context = ssl._create_unverified_context

# set plot style
sns.set_theme()


In [6]:

nltk.download(['punkt','stopwords','wordnet'])


[nltk_data] <urlopen error [Errno 11001] getaddrinfo failed>


False

In [7]:
df = pd.read_csv('Data/processed/train.csv')
df


,headlines,description,content,url,category
0,RBI revises definition of politically-exposed ...,The central bank has also asked chairpersons a...,The Reserve Bank of India (RBI) has changed th...,https://indianexpress.com/article/business/ban...,business
1,NDTV Q2 net profit falls 57.4% to Rs 5.55 cror...,NDTV's consolidated revenue from operations wa...,Broadcaster New Delhi Television Ltd on Monday...,https://indianexpress.com/article/business/com...,business
2,"Akasa Air ‘well capitalised’, can grow much fa...",The initial share sale will be open for public...,Homegrown server maker Netweb Technologies Ind...,https://indianexpress.com/article/business/mar...,business
3,India’s current account deficit declines sharp...,The current account deficit (CAD) was 3.8 per ...,India’s current account deficit declined sharp...,https://indianexpress.com/article/business/eco...,business
4,"States borrowing cost soars to 7.68%, highest ...",The prices shot up reflecting the overall high...,States have been forced to pay through their n...,https://indianexpress.com/article/business/eco...,business
...,...,...,...,...,...
5515,"Samsung sends out invites for ‘Unpacked 2024’,...",Samsung is most likely to announce next-genera...,Samsung plans to reveal the next-generation fl...,https://indianexpress.com/article/technology/t...,technology
5516,Google Pixel 8 Pro accidentally appears on off...,The Pixel 8 Pro will most likely carry over it...,Google once again accidentally gave us a glimp...,https://indianexpress.com/article/technology/m...,technology
5517,Amazon ad on Google Search redirects users to ...,Clicking on the real looking Amazon ad will op...,A new scam seems to be making rounds on the in...,https://indianexpress.com/article/technology/t...,technology
5518,"Elon Musk’s X, previously Twitter, now worth l...","Elon Musk's X, formerly Twitter, has lost more...",More than a year after Elon Musk acquired Twit...,https://indianexpress.com/article/technology/s...,technology


Checking for mising values


In [4]:
#df = df.fillna("missing")


In [8]:
# Summary of the dataset
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5520 entries, 0 to 5519
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   headlines    5520 non-null   object
 1   description  5520 non-null   object
 2   content      5520 non-null   object
 3   url          5520 non-null   object
 4   category     5520 non-null   object
dtypes: object(5)
memory usage: 215.8+ KB
None


In [9]:
# checking for missing values
df.isnull().sum()


headlines      0
description    0
content        0
url            0
category       0
dtype: int64

In [10]:
# checking for duplicates
duplicates = df.duplicated().sum()
duplicates



0

In [11]:
# Initialise the Stemmer
ps = PorterStemmer()



In [13]:

# Preprocessing function
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])
    
    # Tokenize the text
    words = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Apply stemming
    words = [ps.stem(word) for word in words]
    
    # Join words back into a single string
    cleaned_text = ' '.join(words)
    return cleaned_text



In [14]:
# Function to clean the URL
def clean_url(url):
    # Remove 'http://' or 'https://'
    url = re.sub(r'^https?://', '', url)
    
    # Remove fragments (everything after #)
    url = re.sub(r'#.*$', '', url)
    
    # Remove query parameters (everything after ?)
    url = re.sub(r'\?.*$', '', url)
    
    # Extract base domain using urlparse
    parsed_url = urlparse(url)
    domain = parsed_url.netloc  # This extracts the domain (e.g., 'indianexpress.com')
    
    # Extract the article path (e.g., '/article/business/com...')
    path = parsed_url.path
    
    # Combine the domain and the first segment of the path (e.g., 'article')
    # Adjust this if you want to capture more/less of the path
    cleaned_url = domain + path.split('/')[0]
    
    return cleaned_url




In [15]:

# Apply the cleaning function to the 'url' column
df['cleaned_url'] = df['url'].apply(clean_url)
print(df[['url', 'cleaned_url']].head())



                                                 url        cleaned_url
0  https://indianexpress.com/article/business/ban...  indianexpress.com
1  https://indianexpress.com/article/business/com...  indianexpress.com
2  https://indianexpress.com/article/business/mar...  indianexpress.com
3  https://indianexpress.com/article/business/eco...  indianexpress.com
4  https://indianexpress.com/article/business/eco...  indianexpress.com


In [16]:
# Apply cleaning function to the 'headlines' column and print
df['cleaned_headlines'] = df['headlines'].apply(clean_text)
print(df[['headlines', 'cleaned_headlines']].head())


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\true5/nltk_data'
    - 'C:\\Users\\true5\\anaconda3\\envs\\Linear_regression\\nltk_data'
    - 'C:\\Users\\true5\\anaconda3\\envs\\Linear_regression\\share\\nltk_data'
    - 'C:\\Users\\true5\\anaconda3\\envs\\Linear_regression\\lib\\nltk_data'
    - 'C:\\Users\\true5\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
# Apply cleaning function to the 'description' column and print
df['cleaned_description'] = df['description'].apply(clean_text)
print(df[['description', 'cleaned_description']].head())


In [ ]:
# Apply cleaning function to the 'category' column and print
df['cleaned_category'] = df['category'].apply(clean_text)
print(df[['category', 'cleaned_category']].head())


In [ ]:

# Apply cleaning function to the 'content' column and print
df['cleaned_content'] = df['content'].apply(clean_text)
print(df[['content', 'cleaned_content']].head())

In [ ]:
# Print only the cleaned data
print(df[['cleaned_category']].head())
print(df[['cleaned_url']].head())
print(df[['cleaned_content']].head())
print(df[['cleaned_description']].head())
print(df[['cleaned_headlines']].head())